In [1]:
import pandas as pd
import numpy as np
from pandas import Series, DataFrame
import matplotlib.pyplot as plt
import sklearn.preprocessing as preprocessing
from sklearn import linear_model
from pylab import mpl
from sklearn import cross_validation
mpl.rcParams['font.sans-serif'] = ['FangSong'] # 指定默认字体
mpl.rcParams['axes.unicode_minus'] = False # 解决保存图像是负号'-'显示为方块的问题

e:\pycharmproject\mltest\venv\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
figsize=(12,5)
regex_a = "Survived|Age_.*|SibSp|Parch|Fare_.*|Cabin_*|Embarked_.*|Sex_.*|Pclass.*|AgePclass"
data_train = pd.read_csv('titanic_data/train.csv')
scaler = preprocessing.StandardScaler()
data_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


In [3]:
# The model of feture Age
def set_Age_Gap(df):
    df.loc[(df.Age.isnull()),'Age'] = df.Age.mean()
    return df

def set_Age_Discrete(df):
    tmp_age = df.Age
    factor = pd.cut(tmp_age, [0,15,70,100],labels = ['Age_young','Age_adult','Age_old'])
    dummies_age_Discrete = pd.get_dummies(factor)
    return dummies_age_Discrete


In [4]:
# The model of feture Cabin 
def set_Cabin_type(df):
    df.loc[ (df.Cabin.notnull()), 'Cabin' ] = "Yes"
    df.loc[ (df.Cabin.isnull()), 'Cabin' ] = "No"
    return df

In [5]:
# The model of Fare
def set_Fare_scale(df):
    return scaler.fit_transform(df['Fare'].values.reshape(-1,1))

In [6]:
def set_Feature(data_train):
    data_train = set_Age_Gap(data_train)

    data_train = set_Cabin_type(data_train)

    dummies_Cabin = pd.get_dummies(data_train['Cabin'], prefix= 'Cabin')

    dummies_Embarked = pd.get_dummies(data_train['Embarked'], prefix= 'Embarked')

    dummies_Sex = pd.get_dummies(data_train['Sex'], prefix= 'Sex')

    dummies_Pclass = pd.get_dummies(data_train['Pclass'], prefix= 'Pclass')

    b = set_Age_Discrete(data_train)

    df = pd.concat([data_train, dummies_Cabin, dummies_Embarked, dummies_Sex, dummies_Pclass,b], axis=1)

    df['Fare_scaled'] = set_Fare_scale(data_train)

    # New feature AgePclass from 0.77 to 0.775
    df['AgePclass'] = df['Age']*df['Sex_female']
    df.drop(['Pclass', 'Name', 'Sex', 'Ticket', 'Cabin', 'Embarked','Fare','Age'], axis=1, inplace=True)
    
    return df

In [7]:
def get_X_y(df):
    train_np = df.filter(regex=regex_a).values
    
    X = train_np[:,1:]
    
    y = train_np[:,0]
    
    return X,y

In [8]:

df = set_Feature(data_train=data_train)
# 用正则取出我们要的属性值
X,y = get_X_y(df)

# fit到RandomForestRegressor之中
clf = linear_model.LogisticRegression(C=0.5,penalty='l1',tol=1e-6)
clf.fit(X, y)

clf

LogisticRegression(C=0.5, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=None, solver='liblinear', tol=1e-06,
          verbose=0, warm_start=False)

In [10]:
# 对cross validation数据进行预测
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
pipe_titanic = Pipeline([
                        ('clf',LogisticRegression(C=1000.0,penalty='l2',tol=1e-4))])
pipe_titanic.fit(X,y)


Pipeline(memory=None,
     steps=[('clf', LogisticRegression(C=1000.0, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False))])

In [14]:
data_test = pd.read_csv("Titanic_data/test.csv")
data_test.loc[ (data_test.Fare.isnull()), 'Fare' ] = 0
df_test = set_Feature(data_test)
test = df_test.filter(regex=regex_a)
predictions = clf.predict(test)
result = pd.DataFrame({'PassengerId':data_test['PassengerId'].values, 'Survived':predictions.astype(np.int32)})
result.to_csv("Titanic_data/pip.csv", index=False)
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 17 columns):
SibSp          418 non-null int64
Parch          418 non-null int64
Cabin_No       418 non-null uint8
Cabin_Yes      418 non-null uint8
Embarked_C     418 non-null uint8
Embarked_Q     418 non-null uint8
Embarked_S     418 non-null uint8
Sex_female     418 non-null uint8
Sex_male       418 non-null uint8
Pclass_1       418 non-null uint8
Pclass_2       418 non-null uint8
Pclass_3       418 non-null uint8
Age_young      418 non-null uint8
Age_adult      418 non-null uint8
Age_old        418 non-null uint8
Fare_scaled    418 non-null float64
AgePclass      418 non-null float64
dtypes: float64(2), int64(2), uint8(13)
memory usage: 18.4 KB
